<a href="https://colab.research.google.com/github/marcinwolter/DeepLearning_2020/blob/main/mnist_mlp_minimal_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Optuna example that optimizes a neural network classifier configuration for the
MNIST dataset using Keras.
In this example, we optimize the validation accuracy of MNIST classification using
Keras. We optimize the filter and kernel size, kernel stride and layer activation.
"""
! pip install optuna

import warnings

from keras.backend import clear_session
from keras.datasets import mnist
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop

import optuna

In [2]:

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 20


In [3]:
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)[:N_TRAIN_EXAMPLES].astype("float32") / 255
    x_valid = x_valid.reshape(10000, 784)[:N_VALID_EXAMPLES].astype("float32") / 255
    y_train = y_train[:N_TRAIN_EXAMPLES]
    y_valid = y_valid[:N_VALID_EXAMPLES]
    

    units = int(trial.suggest_discrete_uniform("units1", 32,1024,1))
    dropout = trial.suggest_uniform('dropout1', .00,.50)
    act = trial.suggest_categorical('activation', ['relu','sigmoid','linear'])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    epochs = int(trial.suggest_discrete_uniform('epochs', 10,50,1))

    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(784,)))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units, activation='relu'))
    model.add(Dropout(dropout))

    model.add(Dense(CLASSES, activation="softmax"))

    # We compile our model with a sampled learning rate.
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=epochs,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]


# Find the best parameters

In [4]:


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning:

Recent Keras release (2.4.0) simply redirects all APIs in the standalone keras package to point to tf.keras. There is now only one Keras: tf.keras. There may be some breaking changes for some workflows by upgrading to keras 2.4.0. Test before upgrading. REF:https://github.com/keras-team/keras/releases/tag/2.4.0

[I 2020-11-30 23:17:27,273] A new study created in memory with name: no-name-e89ef8f2-c576-4d15-b30b-11c82293a5f0
[I 2020-11-30 23:17:30,828] Trial 0 finished with value: 0.8870000243186951 and parameters: {'units1': 627.0, 'dropout1': 0.4405346229913713, 'activation': 'sigmoid', 'lr': 0.011522109389792864, 'epochs': 17.0}. Best is trial 0 with value: 0.8870000243186951.
[I 2020-11-30 23:17:35,113] Trial 1 finished with value: 0.7369999885559082 and parameters: {'units1': 311.0, 'dropout1': 0.22475898480906414, 'activation': 'relu', 'lr': 0.04477075508632895, 'epochs': 44.0}. Best is trial 0 with valu

In [7]:

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.9399999976158142
  Params: 
    units1: 622.0
    dropout1: 0.4381547142738195
    activation: linear
    lr: 0.0021825108171190195
    epochs: 31.0


## Visualizing the Optimization History

In [8]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

## Visualizing High-dimensional Parameter Relationships

In [9]:
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study)

## Visualizing Parameter Relationships

In [10]:
from optuna.visualization import plot_contour

plot_contour(study)

### Selecting Parameters to Visualize

In [11]:
plot_contour(study, params=['units1', 'dropout1', 'lr','epochs'])

## Visualizing Individual Parameters

In [12]:
from optuna.visualization import plot_slice

plot_slice(study)

### Selecting Parameters to Visualize

In [13]:
plot_slice(study, params=['units1', 'dropout1'])

## Visualizing Parameter Importances

In [14]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)